#correct

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
from collections import OrderedDict

In [ ]:
DIR = "/content/drive/MyDrive/RAW2/"#path
entries = [entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]

In [ ]:
data=[]
def read_text_file(f):
  contents = f.read()
  d=contents.split(" ")
  data.append("FILE_NAME $ "+os.path.basename(f.name)+'')
  #PETITIONER
  petitioner = re.search('PETITIONER:(.*?)(Vs.|RESPONDENT)', contents)
  
  #RESPONDENT
  respondent = re.search('RESPONDENT:(.*?)DATE', contents)

  #DATE OF JUDGMENT
  date =re.search(r'DATE OF JUDGMENT: (\S+)', contents)

  #CASE_NO
  appeal_no = re.search(r'(?=Appeal No)(\w+\W+)*?\d+(\w+\W+)*?\b\d{4}\b', contents,re.IGNORECASE)
  case_no = re.search(r'(?=CASE NO)(\w+\W+)*?\d+(\w+\W+)*?\b\d{4}\b', contents)

  #CASENAME
  if respondent and petitioner and date:
      case_name  = petitioner.group(1) + 'VS.' + respondent.group(1) + ' on '+date.group(1)
      data.append("CASE_NAME $ "+case_name+"")
      data.append("PETITIONER $ "+petitioner.group(1)+"")
      data.append("RESPONDENT $ "+respondent.group(1)+"")
      data.append("DATE_OF_JUDGMENT $ "+date.group(1)+"")
  elif respondent and petitioner:
      case_name  = petitioner.group(1) + 'VS.' + respondent.group(1)
      data.append("CASE_NAME $ "+case_name+"")
      data.append("PETITIONER $ "+petitioner.group(1)+"")
      data.append("RESPONDENT $ "+respondent.group(1)+"")
  
  elif case_no:
      case_name=case_no.group()
      data.append("CASE_NAME $ "+case_name+"")
  elif appeal_no:
      case_name=appeal_no.group()
      data.append("CASE_NAME $ "+case_name+"")
  if appeal_no:
      data.append("APPEAL_NO $ "+appeal_no.group()+"")
  if case_no:
      data.append("CASE_NO $ "+case_no.group()+"")

  #CASE_TYPE
  ctyp=re.search('CIVIL',contents,re.IGNORECASE)
  if ctyp:
      ctyp = ctyp.group()
      data.append("CASE_TYPE $ "+ctyp+'')
  ctyp=re.search('CRIMINAL',contents,re.IGNORECASE)
  if ctyp:
      ctyp = ctyp.group()
      data.append("CASE_TYPE $ "+ctyp+'')

  #AUTHOR
  auth = re.search('BENCH: (.*?) (BENCH)', contents)
  if auth:
      auth = auth.group(1)
      auth = ' '.join(reversed(auth.split(",")))
      if "&" in auth:
          auth = auth.split("&")[0]
      data.append("AUTHOR $ "+auth+"")

  #BENCH

  bench = re.search('BENCH:(.+?)(CITATION:|JUDGMENT|ACT:)', contents)    
  print(bench)
  if bench:
      count=bench.group().count("BENCH:")
      if "&" in bench.group(1) :
          bench = ', '.join((bench.group(1)).split("&"))
          bench=bench.replace('(J)',"")
          bench=bench.replace('(CJ)',"")
          data.append("BENCH $ "+bench+"")
      else:
          names = re.findall(r'((?:\w+), (?:\w+(?: \w+)?))(?= BENCH:| CITATION:| \w+, | JUDGMENT)', bench.group(0))
          print(names)
          names = list(OrderedDict.fromkeys(names))
          print(names)
          names = ', '.join((map(lambda n:' '.join(n.split(', ')[-1::-1]), names)))
          names=names.replace('(CJ)',"")
          names=names.replace('(J)',"")
          print("names")
          if (names):
              data.append("BENCH $ "+names+"")
          else:
            if(count==2):
              temp=bench.group(1)[bench.group(1).rfind("BENCH:")+7:].strip()
              temp=temp.replace('(CJ)',"")
              temp=temp.replace('(J)',"")
              x=temp.split(" ")
              print(x)
              values=""
              t=""
              for x in x:
                if "," in x:
                  values+=t
                  t=x+" "
                else:
                  values+=x+" "
              values+=t
              print(values[:-2])
              data.append("BENCH $ "+values[:-2]+"")
            else:
              data.append("BENCH $ "+bench.group(1)+"")




  #JURISDICTION
  jur = re.search(r"(\w+\W+){1}(JURISDICTION)", contents)
  if jur:
      data.append("JURISDICTION $ "+jur.group(0)+"")

  #COURT TYPE
  jur = re.search('Supreme Court of India', contents,re.IGNORECASE)
  if jur:
      data.append("COURT_TYPE $ "+jur.group()+"")
  jur = re.search('High Court', contents)
  if jur:
      data.append("COURT_TYPE $ "+jur.group()+"")
  jur = re.search('District Court', contents)
  if jur:
      data.append("COURT_TYPE $ "+jur.group()+"")

  #LOCATION
  loc = re.search('(\w{4,}) (High Court)', contents)
  if loc:
      if ((loc.group(1)).lower()=='pradesh'):
          loc = re.search('(\w{4,}) (Pradesh)', contents,re.IGNORECASE)
          data.append("LOCATION $ "+loc.group()+"")
      elif ((loc.group(1)).lower()=='kashmir'):
          data.append("LOCATION $ Jammu and Kashmir"+'')
      elif ((loc.group(1)).lower()=='haryana'):
          data.append("LOCATION $ Punjab and Haryana"+'')
      else:
          data.append("LOCATION $ "+loc.group(1)+"")

  #COURT DECISION
  positives=['allowed','allow','accept']
  negatives=['disposed','disallowed','remitted','dismissed','remit','dismiss','dismissal']
  length= len(d)-1
  #print(d[length])
  for i in range(length,-1,-1):
    dec=re.sub(r'[^\w\s]', '', d[i]).lower()
    #print(dec)
    #dec=re.search(i,contents,re.IGNORECASE)
    if dec in positives:
      data.append("COURT_DECISION$accepted")
      break
    if dec in negatives:
      data.append("COURT_DECISION$rejected")
      break
 


In [ ]:
for entry in entries:
  f=open(DIR+entry, "r")
  read_text_file(f)

<re.Match object; span=(122, 177), match='BENCH: Syed Shah Mohammed Quadri & S.N. Phukan JU>
<re.Match object; span=(113, 182), match='BENCH: THOMAS K.T. (J) BENCH: THOMAS K.T. (J) PUN>
[]
[]
names
['THOMAS', 'K.T.', '', 'PUNCHHI,', 'M.M.']
THOMAS K.T.  M.M. PUNCHHI
<re.Match object; span=(136, 180), match='BENCH: J.S VERMA & K.S. PARIPOORNAN JUDGMENT'>
<re.Match object; span=(183, 221), match='BENCH: M.B. SHAH & R.P. SETHI JUDGMENT'>
<re.Match object; span=(104, 115), match='BENCH: ACT:'>
[]
[]
names
<re.Match object; span=(87, 180), match='BENCH: KAPUR, J.L. BENCH: KAPUR, J.L. GAJENDRAGAD>
[]
[]
names
['KAPUR,', 'J.L.', 'GAJENDRAGADKAR,', 'P.B.', 'AIYYAR,', 'T.L.', 'VENKATARAMA']
J.L. KAPUR, P.B. GAJENDRAGADKAR, T.L. VENKATARAMA AIYYAR
<re.Match object; span=(171, 217), match='BENCH: SUJATA V. MANOHAR, G.B. PATTANAIK. ACT:'>
[]
[]
names
<re.Match object; span=(116, 194), match='BENCH: BACHAWAT, R.S. BENCH: BACHAWAT, R.S. WANCH>
[]
[]
names
['BACHAWAT,', 'R.S.', 'WANCHOO,', 'K.N.', 'S

In [ ]:
print(data)

['FILE_NAME $ 1397402.txt', 'CASE_NAME $  GORDHAN DAS VS. PIRKHAN & ORS.  on 27/11/2001', 'PETITIONER $  GORDHAN DAS ', 'RESPONDENT $  PIRKHAN & ORS. ', 'DATE_OF_JUDGMENT $ 27/11/2001', 'APPEAL_NO $ Appeal No.219 of 1975', 'CASE_NO $ CASE NO.: Appeal (civil) 4391 of 1990', 'CASE_TYPE $ civil', 'BENCH $  Syed Shah Mohammed Quadri ,  S.N. Phukan ', 'COURT_TYPE $ High Court', 'LOCATION $ Madhya Pradesh', 'COURT_DECISION$accepted', 'FILE_NAME $ 156593612.txt', 'CASE_NAME $  HEIRS OF VRAJLAL J. GANATRA VS. HAIRS OF PARSHOTTAM S. SHAH  on 30/04/1996', 'PETITIONER $  HEIRS OF VRAJLAL J. GANATRA ', 'RESPONDENT $  HAIRS OF PARSHOTTAM S. SHAH ', 'DATE_OF_JUDGMENT $ 30/04/1996', 'AUTHOR $ THOMAS K.T. (J)', 'BENCH $ THOMAS K.T.  M.M. PUNCHHI', 'COURT_TYPE $ High Court', 'COURT_DECISION$rejected', 'FILE_NAME $ 298453.txt', 'CASE_NAME $  RAGHBIR SINGH VS. SURJIT SINGH AND ORS. ETC. ETC.  on 22/08/1994', 'PETITIONER $  RAGHBIR SINGH ', 'RESPONDENT $  SURJIT SINGH AND ORS. ETC. ETC. ', 'DATE_OF_JUDGME

In [ ]:
import pandas as pd
tok_ent = pd.DataFrame(data,columns =['data'])
tok_ent[['Token','Entity']] = tok_ent.data.str.split("$",expand=True)
#print(tok_ent[0:60])
t = pd.DataFrame(tok_ent['data'])
t
#tok_ent['data']

,data
0,FILE_NAME $ 1397402.txt
1,CASE_NAME $ GORDHAN DAS VS. PIRKHAN & ORS. o...
2,PETITIONER $ GORDHAN DAS
3,RESPONDENT $ PIRKHAN & ORS.
4,DATE_OF_JUDGMENT $ 27/11/2001
...,...
537,AUTHOR $ E.S. (J) VENKATARAMIAH
538,BENCH $ E.S. VENKATARAMIAH
539,JURISDICTION $ ORIGINAL JURISDICTION
540,COURT_TYPE $ High Court


In [ ]:
ids_dict={}
for i in range(tok_ent.shape[0]):
  temp=tok_ent['Token'][i]

  if 'DATE_OF_JUDGMENT' in temp:
    key=int(tok_ent['Entity'][i][-4:]+tok_ent['Entity'][i][-7:-5])
    if not (key in ids_dict):
      ids_dict[key] = 0
print(ids_dict)

{200111: 0, 199604: 0, 199408: 0, 200110: 0, 196305: 0, 196203: 0, 199812: 0, 196608: 0, 200712: 0, 200804: 0, 200109: 0, 199708: 0, 195410: 0, 195212: 0, 199609: 0, 199608: 0, 200704: 0, 195303: 0, 199702: 0, 195309: 0, 200708: 0, 199803: 0, 196011: 0, 200108: 0, 196709: 0, 196609: 0, 199610: 0, 200801: 0, 200803: 0, 200702: 0, 199603: 0, 199602: 0, 195510: 0, 195902: 0, 199711: 0, 199512: 0, 199404: 0, 199004: 0, 199412: 0, 197608: 0, 199312: 0, 199502: 0, 198911: 0, 199310: 0, 199104: 0, 198406: 0}


In [ ]:
ids=[]
for i in range(tok_ent.shape[0]):
  temp=tok_ent['Token'][i]

  if 'DATE_OF_JUDGMENT' in temp:
    key=int(tok_ent['Entity'][i][-4:]+tok_ent['Entity'][i][-7:-5])
    ids_dict[key] = ids_dict[key]+1
    val= ids_dict[key]
    ids.append(str(key)+'KS'+str(val)+'SC')
    
print(ids)

['200111KS1SC', '199604KS1SC', '199408KS1SC', '200110KS1SC', '196305KS1SC', '196203KS1SC', '199812KS1SC', '196608KS1SC', '200712KS1SC', '200804KS1SC', '200109KS1SC', '199708KS1SC', '195410KS1SC', '195212KS1SC', '199609KS1SC', '199608KS1SC', '200704KS1SC', '195303KS1SC', '199702KS1SC', '195309KS1SC', '199608KS2SC', '200708KS1SC', '199803KS1SC', '199812KS2SC', '199604KS2SC', '196011KS1SC', '200108KS1SC', '196709KS1SC', '196609KS1SC', '199610KS1SC', '200801KS1SC', '200803KS1SC', '200702KS1SC', '199603KS1SC', '199602KS1SC', '195510KS1SC', '195902KS1SC', '200801KS2SC', '199711KS1SC', '199512KS1SC', '199404KS1SC', '199004KS1SC', '199412KS1SC', '197608KS1SC', '199312KS1SC', '199502KS1SC', '198911KS1SC', '199310KS1SC', '199104KS1SC', '198406KS1SC']


In [ ]:
len(ids)

50

In [ ]:
re=""
index=0
for i in range(tok_ent.shape[0]):
  temp=tok_ent['Token'][i]

  #if 'CASE_NAME' in temp:
    #CASENAME=tok_ent['Entity'][i]
  
  
  if 'FILE_NAME' in temp:
      re+='\n'+tok_ent['Entity'][i]+'\n'
      CASENAME=ids[index]
      re+='CASE hasCaseId '+ CASENAME+'\n'
      index+=1
      
  if 'DATE_OF_JUDGMENT' in temp:
      re+=CASENAME+' hasDate '+tok_ent['Entity'][i]+'\n'
      re+=CASENAME+' hasYear '+tok_ent['Entity'][i][-4:]+'\n'
      re+=CASENAME+' hasMonth '+tok_ent['Entity'][i][-7:-5]+'\n'
      
      
  if 'CASE_NAME' in temp:
      re+=CASENAME+' hasCaseName '+tok_ent['Entity'][i]+'\n'


  if 'PETITIONER' in temp:
      re+=CASENAME+' hasParty '+ tok_ent['Token'][i]+'\n'
      re+='PETITIONER hasName '+tok_ent['Entity'][i]+'\n'

  if 'RESPONDENT' in temp:
      re+=CASENAME+' hasParty '+ tok_ent['Token'][i]+'\n'
      re+='RESPONDENT hasName '+tok_ent['Entity'][i]+'\n'

  if 'CASE_NO' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCaseNo '+ tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCaseNo '+ tok_ent['Entity'][i]+'\n'

  if 'CASE_TYPE' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCaseType '+ tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCaseType '+ tok_ent['Entity'][i]+'\n'
  
  if 'APPEAL_NO' in temp:
      if(CASENAME):
          re+=CASENAME+' hasAppealNo '+ tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasAppealNo '+ tok_ent['Entity'][i]+'\n'

  
  if 'AUTHOR' in temp:
      re+=CASENAME+' hasAuthor '+tok_ent['Entity'][i]+'\n'

  if 'BENCH' in temp:
      re+=CASENAME+' hasCourtOfficial Judge '+'\n'
      str=tok_ent['Entity'][i]
      my_list =str.split(",")
      for x in range(len(my_list)):
          re+='Judge hasName '+my_list[x]+'\n'

  if 'JURISDICTION' in temp:
      if(CASENAME):
          re+=CASENAME+' hasJurisdiction '+tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasJurisdiction '+tok_ent['Entity'][i]+'\n'
  
  if 'COURT_TYPE' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCourt '+tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCourt '+tok_ent['Entity'][i]+'\n'
      
  if 'LOCATION' in temp:
      re+=tok_ent['Entity'][i-1]+' hasLocation '+tok_ent['Entity'][i]+'\n'

  if 'COURT_DECISION' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCourtDecision '+tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCourtDecision '+tok_ent['Entity'][i]+'\n'
    

In [ ]:
print(re)


 1397402.txt
CASE hasCaseId 200111KS1SC
200111KS1SC hasCaseName   GORDHAN DAS VS. PIRKHAN & ORS.  on 27/11/2001
200111KS1SC hasParty PETITIONER 
PETITIONER hasName   GORDHAN DAS 
200111KS1SC hasParty RESPONDENT 
RESPONDENT hasName   PIRKHAN & ORS. 
200111KS1SC hasDate  27/11/2001
200111KS1SC hasYear 2001
200111KS1SC hasMonth 11
200111KS1SC hasAppealNo  Appeal No.219 of 1975
200111KS1SC hasCaseNo  CASE NO.: Appeal (civil) 4391 of 1990
200111KS1SC hasCaseType  civil
200111KS1SC hasCourtOfficial Judge 
Judge hasName   Syed Shah Mohammed Quadri 
Judge hasName   S.N. Phukan 
200111KS1SC hasCourt  High Court
 High Court hasLocation  Madhya Pradesh
200111KS1SC hasCourtDecision accepted

 156593612.txt
CASE hasCaseId 199604KS1SC
199604KS1SC hasCaseName   HEIRS OF VRAJLAL J. GANATRA VS. HAIRS OF PARSHOTTAM S. SHAH  on 30/04/1996
199604KS1SC hasParty PETITIONER 
PETITIONER hasName   HEIRS OF VRAJLAL J. GANATRA 
199604KS1SC hasParty RESPONDENT 
RESPONDENT hasName   HAIRS OF PARSHOTTAM S. SHAH 
1

#try

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
keys=[1952,
1953,
1953,
1954,
1955,
1959,
1960,
1962,
1963,
1966,
1966,
1967,
1976,
1984,
1989,
1990,
1991,
1993,
1993,
1994,
1994,
1994,
1995,
1995,
1996,
1996,
1996,
1996,
1996,
1996,
1997,
1997,
1997,
1998,
1998,
2001,
2001,
2001,
2001,
2007,
2007,
2007,
2007,
2008,
2008,
2008]

keys_2=[12,
3,
9,
10,
10,
2,
11,
3,
5,
8,
9,
9,
8,
6,
11,
4,
4,
10,
12,
8,
12,
4,
12,
2,
9,
2,
8,
10,
3,
4,
2,
11,
8,
3,
12,
11,
9,
10,
8,
4,
12,
2,
8,
1, 
4,
3]

In [ ]:
main_keys=[]
for i in range(len(keys)):
  if keys_2[i]<10:
    main_keys.append(int(str(keys[i])+str(0)+str(keys_2[i])))
  else:
    main_keys.append(int(str(keys[i])+str(keys_2[i])))
print(main_keys)
id_check= dict.fromkeys(main_keys, 0)
print(id_check)

In [ ]:
import os
import re
from collections import OrderedDict

In [ ]:
DIR = "/content/drive/MyDrive/Indian Legal Dataset/"#folder path containing case files
#https://drive.google.com/drive/folders/1tJFfZOhraH02ok_k2IGUDTMoguxyED_Y?usp=sharing
#use the above given path to add the above file in your drive and add the path in the above dir


# if you want to list all the contents in DIR
#entries = [entry for entry in os.listdir(DIR)]

# if you want to list all the files in DIR
entries = [entry for entry in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, entry))]


FileNotFoundError: ignored

In [ ]:
data= ""

In [ ]:
judges_name=['N.V. Ramana', 'Harilal Jekisundas Kania', 'M. Patanjali Sastri', 'Mehr Chand Mahajan', 'Bijan Kumar Mukherjea', 'Sudhi Ranjan Das', 'Bhuvneshwar Prasad Sinha', 'P.B. Gajendragadkar', 'A.K. Sarkar', 'K. Subba Rao', 'K.N. Wanchoo', 'M. Hidayatullah', 'J.C. Shah', 'S.M. Sikri', 'A.N. Ray', 'M. Hameedullah Beg', 'Y.V. Chandrachud', 'P.N. Bhagwati', 'R.S. Pathak', 'E.S. Venkataramiah', 'Sabyasachi Mukherjee', 'Ranganath Misra', 'K.N. Singh', 'M.H. Kania', 'L.M. Sharma', 'M.N. Venkatachaliah', 'A.M. Ahmadi', 'J.S. Verma', 'M.M. Punchhi', 'A.S. Anand', 'S.P. Bharucha', 'B.N. Kirpal', '.B. Patnaik', 'V.N. Khare', 'S. Rajendra Babu', 'R.C. Lahoti', 'Y.K. Sabharwal', 'K.G. Balakrishnan', 'S.H. Kapadia ', 'Altamas Kabir ', 'P. Sathasivam', 'R. M. Lodha', 'H.L. Dattu', 'T.S. Thakur', 'Jagdish Singh Khehar', 'Dipak Misra', 'Ranjan Gogoi', 'Sharad Arvind Bobde', 'Sir Saiyid Fazl Ali', 'N. Chandrasekhara Aiyar', 'Vivian Bose ', 'Ghulam Hasan', 'Natwarlal Harilal Bhagwati', 'B. Jagannadhadas', 'T.L Venkatarama Aiyyar', 'Syed Jaffer Imam', 'S.K Das', 'P. Govinda Menon', 'J.L. Kapur', 'K.C. Das Gupta', 'Raghubar Dayal', 'N. Rajagopala Ayyangar', 'J.R. Mudholkar', 'R.S. Bachawat', 'V. Ramaswami', 'P. Satyanarayana Raju', 'J.M. Shelat', 'Vishishtha Bhargava', 'G.K. Mitter ', 'C.A. Vaidyialingam', 'K.S. Hegde', 'A.N. Grover ', 'P. Jaganmohan Reddy', 'I.D. Dua', 'Subimal Chandra Roy', 'D.G. Palekar', 'Hans Raj Khanna', 'Kuttyil Kurien Mathew', 'S.N. Dwivedi', 'A.K. Mukherjea', 'A Alagiriswami', 'V.R. Krishna Iyer ', 'P.K. Goswami', 'R.S. Sarkaria', 'A.C. Gupta', 'N.L. Untwalia', 'S. Murtaza Fazal Ali', 'P.N. Shingal', 'Jaswant Singh', 'P.S. Kailasam', 'V.D. Tulzapurkar', 'D.A. Desai', 'A.D. Koshal', 'O. Chinnappa Reddy', 'A.P. Sen', 'Baharul Islam', 'A. Varadarajan', 'Amarendra Nath Sen', 'V. Balakrishna Eradi', 'R.B. Misra', 'D.P. Madon', 'M.P. Thakkar', 'V. Khalid', 'G.L. Oza', 'B.C. Ray', 'M.M. Dutt', 'S. Natarajan', 'K. Jagannatha Shetty', 'S. Ranganathan', 'D.N. Ojha', 'S. Ratnavel Pandian', 'T.K. Thommen ', 'K.N. Saikia', 'Kuldip Singh', 'V. Ramaswami', 'P.B. Sawant', 'N.M. Kasliwal', 'K. Ramaswamy', 'M. Fathima Beevi', 'K. Jayachandra Reddy', 'S.C. Agrawal', 'R.M. Sahai', 'Yogeshwar Dayal', 'S. Mohan', 'B.P. Jeevan Reddy', 'G.N. Ray', 'R.C. Patnaik', 'N.P. Singh', 'N. Venkatachala', 'M.K. Mukherjee', 'Faizanuddin ', 'B.L. Hansaria', 'S.C. Sen', 'K.S. Paripoornan', 'S.B. Majmudar', 'Sujata V Manohar', 'G.T. Nanavati', 'S. Saghir Ahmad', 'K. Venkataswami', 'S.P. Kurdukar', 'K.T. Thomas', 'M. Jagannadha Rao', 'D.P. Wadhwa', 'M. Srinivasan', 'Ajay Prakash Misra', 'S.S.M. Quadri', 'M.B. Shah', 'D.P. Mohapatra', 'U.C. Banerjee', 'N. Santosh Hegde', 'R.P. Sethi', 'S.N. Phukan', 'Doraiswamy Raju', ' Ruma Pal', 'S.N. Variava ', 'Shivaraj V Patil', 'Brijesh Kumar', 'B.N. Agrawal ', 'P. Venkatarama Reddi', 'Ashok Bhan', 'Arijit Pasayat ', 'B.P. Singh', 'D.M. Dharmadhikari', 'H.K. Sema', 'S.B. Sinha ', 'Arun Kumar', 'B.N. Srikrishna', 'A.R. Lakshmanan', 'G.P. Mathur', 'A.K. Mathur', 'C.K. Thakker', 'P.K. Balasubramanyan', 'P.P. Naolekar', 'Tarun Chatterjee ', 'Lokeshwar Singh Panta ', 'B. Sudershan Reddy ', 'V.S. Sirpurkar ', 'H.S. Bedi ', 'Mukundakam Sharma ', 'Markandey Katju ', 'J.M. Panchal ', 'R.V. Raveendran ', 'Cyriac Joseph ', 'A.K. Ganguly ', 'Dalveer Bhandari ', 'Deepak Verma ', 'Swatanter Kumar ', 'D.K. Jain ', 'Aftab Alam ', 'G.S. Singhvi ', 'H.L. Gokhale ', ' Gyan Sudha Misra ', 'K.S. Panicker Radhakrishnan ', 'A.K. Patnaik ', 'S.S. Nijjar ', 'B.S. Chauhan', 'C.K. Prasad ', ' Ranjana Prakash Desai ', 'S.J. Mukhopadhaya ', 'Vikramajit Sen', 'M.Y. Eqbal ', 'Fakkir Mohamed Ibrahim Kalifulla', 'Chockalingam Nagappan', 'V. Gopala Gowda', 'Shiva Kirti Singh', 'Anil R. Dave', 'Pinaki Chandra Ghose', 'Prafulla Chandra Pant', 'Amitava Roy', 'R.K. Agrawal', 'Jasti Chelameswar', 'Adarsh Kumar Goel', 'Kurian Joseph', 'Madan Bhimarao Lokur', 'Arjan Kumar Sikri', 'Abhay Manohar Sapre', 'Deepak Gupta', ' R. Banumathi', 'Arun Mishra', 'Indu Malhotra', 'Mohan M. Shantanagoudar', 'Ashok Bhushan', 'R.F. Nariman', 'Navin Sinha', 'R.Subhash Reddy', 'Vineet Saran', 'L. Nageswara Rao']

In [ ]:
data= ""
with open('3498.txt', 'r') as f:
  contents = f.read()
  data +=" \n"
  data+="FILE_NAME - "+os.path.basename(f.name)+'\n'
  #PETITIONER
  petitioner = re.search('PETITIONER:(.*?)(Vs.|RESPONDENT)', contents)
  
  #RESPONDENT
  respondent = re.search('RESPONDENT:(.*?)DATE', contents)

  #DATE OF JUDGMENT
  date =re.search(r'DATE OF JUDGMENT: (\S+)', contents)

  #CASE_NO
  appeal_no = re.search(r'(?=Appeal No)(\w+\W+)*?\d+(\w+\W+)*?\b\d{4}\b', contents,re.IGNORECASE)
  case_no = re.search(r'(?=CASE NO)(\w+\W+)*?\d+(\w+\W+)*?\b\d{4}\b', contents)

  #CASENAME
  if respondent and petitioner and date:
      case_name  = petitioner.group(1) + 'VS.' + respondent.group(1) + ' on '+date.group(1)
      data+="CASE_NAME - "+case_name+"\n"
      data+="PETITIONER - "+petitioner.group(1)+"\n"
      data+="RESPONDENT - "+respondent.group(1)+"\n"
      data+="DATE_OF_JUDGMENT - "+date.group(1)+"\n"
  elif respondent and petitioner:
      case_name  = petitioner.group(1) + 'VS.' + respondent.group(1)
      data+="CASE_NAME - "+case_name+"\n"
      data+="PETITIONER - "+petitioner.group(1)+"\n"
      data+="RESPONDENT - "+respondent.group(1)+"\n"
  
  elif case_no:
      case_name=case_no.group()
      data+="CASE_NAME - "+case_name+"\n"
  elif appeal_no:
      case_name=appeal_no.group()
      data+="CASE_NAME - "+case_name+"\n"
  if appeal_no:
      data+="APPEAL_NO - "+appeal_no.group()+"\n"
  if case_no:
      data+="CASE_NO - "+case_no.group()+"\n"

  #CASE_TYPE
  ctyp=re.search('CIVIL',contents,re.IGNORECASE)
  if ctyp:
      ctyp = ctyp.group()
      data+="CASE_TYPE - "+ctyp+'\n'
  ctyp=re.search('CRIMINAL',contents,re.IGNORECASE)
  if ctyp:
      ctyp = ctyp.group()
      data+="CASE_TYPE - "+ctyp+'\n'

  #AUTHOR
  auth = re.search('BENCH: (.*?) (BENCH)', contents)
  if auth:
      auth = auth.group(1)
      auth = ' '.join(reversed(auth.split(",")))
      if "&" in auth:
          auth = auth.split("&")[0]
      data+="AUTHOR - "+auth+"\n"

  #BENCH
  bench = re.search('BENCH:(.+?)(CITATION:|JUDGMENT|ACT:)', contents)
  if bench:
      if "&" in bench.group(1) :
          bench = ', '.join((bench.group(1)).split("&"))
          data+="BENCH - "+bench+"\n"
      else:
          names = re.findall(r'((?:\w+), (?:\w+(?: \w+)?))(?= BENCH:| CITATION:| \w+, | JUDGMENT)', bench.group(0))
          names = list(OrderedDict.fromkeys(names))
          names = ', '.join((map(lambda n:' '.join(n.split(', ')[-1::-1]), names)))
          if (names):
              data+="BENCH - "+names+"\n"
          else:
              data+="BENCH - "+bench.group(1)+"\n"


  #JURISDICTION
  jur = re.search(r"(\w+\W+){1}(JURISDICTION)", contents)
  if jur:
      data+="JURISDICTION - "+jur.group(0)+"\n"

  #COURT TYPE
  jur = re.search('Supreme Court of India', contents,re.IGNORECASE)
  if jur:
      data+="COURT_TYPE - "+jur.group()+"\n"
  jur = re.search('High Court', contents)
  if jur:
      data+="COURT_TYPE - "+jur.group()+"\n"
  jur = re.search('District Court', contents)
  if jur:
      data+="COURT_TYPE - "+jur.group()+"\n"

  #LOCATION
  loc = re.search('(\w{4,}) (High Court)', contents)
  if loc:
      if ((loc.group(1)).lower()=='pradesh'):
          loc = re.search('(\w{4,}) (Pradesh)', contents,re.IGNORECASE)
          data+="LOCATION - "+loc.group()+"\n"
      elif ((loc.group(1)).lower()=='kashmir'):
          data+="LOCATION - Jammu and Kashmir"+'\n'
      elif ((loc.group(1)).lower()=='haryana'):
          data+="LOCATION - Punjab and Haryana"+'\n'
      else:
          data+="LOCATION - "+loc.group(1)+"\n"

  #COURT DECISION
  positives=['allowed','allow']
  negatives=['disposed','disallowed','remitted','dismissed','remit']
  length= len(d)-1
  print(d[length])
  for i in range(length,-1,-1):
    dec=re.sub(r'[^\w\s]', '', d[i]).lower()
    print(dec)
    #dec=re.search(i,contents,re.IGNORECASE)
    if dec in positives:
        data.append("COURT_DECISION - Appeal accepted")
        break
    if dec in negatives:
      data.append("COURT_DECISION - Appeal rejected")
      break


FileNotFoundError: ignored

In [ ]:
data=[]
with open('1314.txt', 'r') as f:
  contents = f.read()
  print(contents)
  d=contents.split(" ")
  print(d)
  data.append("FILE_NAME - "+os.path.basename(f.name)+'')
  #PETITIONER
  petitioner = re.search('PETITIONER:(.*?)(Vs.|RESPONDENT)', contents)
  
  #RESPONDENT
  respondent = re.search('RESPONDENT:(.*?)DATE', contents)

  #DATE OF JUDGMENT
  date =re.search(r'DATE OF JUDGMENT: (\S+)', contents)

  #CASE_NO
  appeal_no = re.search(r'(?=Appeal No)(\w+\W+)*?\d+(\w+\W+)*?\b\d{4}\b', contents,re.IGNORECASE)
  case_no = re.search(r'(?=CASE NO)(\w+\W+)*?\d+(\w+\W+)*?\b\d{4}\b', contents)

  #CASENAME
  if respondent and petitioner and date:
      case_name  = petitioner.group(1) + 'VS.' + respondent.group(1) + ' on '+date.group(1)
      data.append("CASE_NAME - "+case_name+"")
      data.append("PETITIONER - "+petitioner.group(1)+"")
      data.append("RESPONDENT - "+respondent.group(1)+"")
      data.append("DATE_OF_JUDGMENT - "+date.group(1)+"")
  elif respondent and petitioner:
      case_name  = petitioner.group(1) + 'VS.' + respondent.group(1)
      data.append("CASE_NAME - "+case_name+"")
      data.append("PETITIONER - "+petitioner.group(1)+"")
      data.append("RESPONDENT - "+respondent.group(1)+"")
  
  elif case_no:
      case_name=case_no.group()
      data.append("CASE_NAME - "+case_name+"")
  elif appeal_no:
      case_name=appeal_no.group()
      data.append("CASE_NAME - "+case_name+"")
  if appeal_no:
      data.append("APPEAL_NO - "+appeal_no.group()+"")
  if case_no:
      data.append("CASE_NO - "+case_no.group()+"")

  #CASE_TYPE
  ctyp=re.search('CIVIL',contents,re.IGNORECASE)
  if ctyp:
      ctyp = ctyp.group()
      data.append("CASE_TYPE - "+ctyp+'')
  ctyp=re.search('CRIMINAL',contents,re.IGNORECASE)
  if ctyp:
      ctyp = ctyp.group()
      data.append("CASE_TYPE - "+ctyp+'')

  #AUTHOR
  auth = re.search('BENCH: (.*?) (BENCH)', contents)
  if auth:
      auth = auth.group(1)
      auth = ' '.join(reversed(auth.split(",")))
      if "&" in auth:
          auth = auth.split("&")[0]
      auth=auth.replace('(CJ)',"")
      auth=auth.replace('(J)',"")
      data.append("AUTHOR - "+auth+"")

  #BENCH

  bench = re.search('BENCH:(.+?)(CITATION:|JUDGMENT|ACT:)', contents)    
  print(bench)
  if bench:


      count=bench.group().count("BENCH:")
      if "&" in bench.group(1) :
          bench = ', '.join((bench.group(1)).split("&"))
          bench=bench.replace('(J)',"")
          bench=bench.replace('(CJ)',"")
          data.append("BENCH - "+bench+"")
      else:
          names = re.findall(r'((?:\w+), (?:\w+(?: \w+)?))(?= BENCH:| CITATION:| \w+, | JUDGMENT)', bench.group(0))
          print(names)
          names = list(OrderedDict.fromkeys(names))
          print(names)
          names = ', '.join((map(lambda n:' '.join(n.split(', ')[-1::-1]), names)))
          names=names.replace('(CJ)',"")
          names=names.replace('(J)',"")
          print("names")
          if (names):
              data.append("BENCH - "+names+"")
          else:
            if(count==2):
              temp=bench.group(1)[bench.group(1).rfind("BENCH:")+7:].strip()
              temp=temp.replace('(CJ)',"")
              temp=temp.replace('(CJ)',"")
              x=temp.split(" ")
              print(x)
              values=""
              t=""
              for x in x:
                if "," in x:
                  values+=t
                  t=x+" "
                else:
                  values+=x+" "
              values+=t
              print(values[:-2])
              data.append("BENCH - "+values[:-2]+"")
            else:
              data.append("BENCH - "+bench.group(1)+"")




  #JURISDICTION
  jur = re.search(r"(\w+\W+){1}(JURISDICTION)", contents)
  if jur:
      data.append("JURISDICTION - "+jur.group(0)+"")

  #COURT TYPE
  jur = re.search('Supreme Court of India', contents,re.IGNORECASE)
  if jur:
      data.append("COURT_TYPE - "+jur.group()+"")
  jur = re.search('High Court', contents)
  if jur:
      data.append("COURT_TYPE - "+jur.group()+"")
  jur = re.search('District Court', contents)
  if jur:
      data.append("COURT_TYPE - "+jur.group()+"")

  #LOCATION
  loc = re.search('(\w{4,}) (High Court)', contents)
  if loc:
      if ((loc.group(1)).lower()=='pradesh'):
          loc = re.search('(\w{4,}) (Pradesh)', contents,re.IGNORECASE)
          data.append("LOCATION - "+loc.group()+"")
      elif ((loc.group(1)).lower()=='kashmir'):
          data.append("LOCATION - Jammu and Kashmir"+'')
      elif ((loc.group(1)).lower()=='haryana'):
          data.append("LOCATION - Punjab and Haryana"+'')
      else:
          data.append("LOCATION - "+loc.group(1)+"")

  #COURT DECISION

  positives=['allowed','allow']
  negatives=['disposed','disallowed','remitted','dismissed','remit']
  length= len(d)-1
  print(d[length])
  for i in range(length,-1,-1):
    dec=re.sub(r'[^\w\s]', '', d[i]).lower()
    print(dec)
    #dec=re.search(i,contents,re.IGNORECASE)
    if dec in positives:
        data.append("COURT_DECISION - accepted")
        break
    if dec in negatives:
      data.append("COURT_DECISION - rejected")
      break
        

PETITIONER: SHIV SAGAR TIWARI Vs. RESPONDENT: UNION OF INDIA & ORS. DATE OF JUDGMENT: 08/11/1996 BENCH: KULDIP SINGH, B.L. HANSARIA ACT: HEADNOTE: JUDGMENT: O R D E R One of the issues for consideration before this Court in this public interest petition is the validity of the allotments of 52 shops/stalls made by Smt. Shiela Kaul the then minister for Housing land Urban development, Government of India. While monitoring this case, this court has passed various interim orders from time to time. It would be useful to quote three such orders. The relevant part of the order dated April 17, 1996 is as under: "The material placed before us inter alia, discloses that 41 shops\stalls allotted on July 3, 1995 were in total contravention of the rules/policy approved situated in Lodhi Road Complex - I & II. Hanuman Road Market. Baba Kharak Singh Marg market, DIZ area Market, and Pleasure Garden near Lajpat Nagar Market. The material on the record further shows that the said 43 shops/stalls were a

1511626


In [ ]:
data

['FILE_NAME - 1314.txt',
 'CASE_NAME -  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996',
 'PETITIONER -  SHIV SAGAR TIWARI ',
 'RESPONDENT -  UNION OF INDIA & ORS. ',
 'DATE_OF_JUDGMENT - 08/11/1996',
 'CASE_TYPE - criminal',
 'BENCH -  KULDIP SINGH, B.L. HANSARIA ']

In [ ]:
import pandas as pd
tok_ent = pd.DataFrame(data,columns =['data'])


In [ ]:
tok_ent[['Token','Entity']] = tok_ent.data.apply(lambda x: pd.Series(str(x).split("-")))
print(tok_ent)

                                                data              Token  \
0                               FILE_NAME - 1314.txt         FILE_NAME    
1  CASE_NAME -  SHIV SAGAR TIWARI VS. UNION OF IN...         CASE_NAME    
2                   PETITIONER -  SHIV SAGAR TIWARI         PETITIONER    
3               RESPONDENT -  UNION OF INDIA & ORS.         RESPONDENT    
4                      DATE_OF_JUDGMENT - 08/11/1996  DATE_OF_JUDGMENT    
5                               CASE_TYPE - criminal         CASE_TYPE    
6              BENCH -  KULDIP SINGH, B.L. HANSARIA              BENCH    

                                              Entity  
0                                           1314.txt  
1    SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS. ...  
2                                 SHIV SAGAR TIWARI   
3                             UNION OF INDIA & ORS.   
4                                         08/11/1996  
5                                           criminal  
6             

In [ ]:
re=""
for i in range(tok_ent.shape[0]):
  temp=tok_ent['Token'][i]
  if 'FILE_NAME' in temp:
    re=re+'hellpo'
    print(i)
  print(temp)
  if 'CASE_NAME' in tok_ent['Token'][i]:
    CASENAME=tok_ent['Entity'][i]
    print(i)

  

0
FILE_NAME 
CASE_NAME 
1
PETITIONER 
RESPONDENT 
DATE_OF_JUDGMENT 
CASE_TYPE 
BENCH 


In [ ]:
print(re)

hellpo


In [ ]:
re=""

for i in range(tok_ent.shape[0]):
  temp=tok_ent['Token'][i]

  if 'CASE_NAME' in temp:
    CASENAME=tok_ent['Entity'][i]
  
  
  if 'FILE_NAME' in temp:
      re+='\n'+tok_ent['Entity'][i]+'\n'
      CASENAME=""
  if 'DATE_OF_JUDGMENT' in temp:
      re+=CASENAME+' hasDate '+tok_ent['Entity'][i]+'\n'
      re+=CASENAME+' hasYear '+str(tok_ent['Entity'][i])[-4:]+'\n'
      re+=CASENAME+' hasMonth '+str(tok_ent['Entity'][i])[-7:-5]+'\n'
      
  if 'CASE_NAME' in temp:
      re+='CASE hasCaseName '+ tok_ent['Entity'][i]+'\n'

  if 'PETITIONER' in temp:
      re+=CASENAME+' hasParty '+ tok_ent['Token'][i]+'\n'
      re+='PETITIONER hasName '+tok_ent['Entity'][i]+'\n'

  if 'RESPONDENT' in temp:
      re+=CASENAME+' hasParty '+ tok_ent['Token'][i]+'\n'
      re+='RESPONDENT hasName '+tok_ent['Entity'][i]+'\n'

  if 'CASE_NO' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCaseNo '+ tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCaseNo '+ tok_ent['Entity'][i]+'\n'

  if 'CASE_TYPE' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCaseType '+ tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCaseType '+ tok_ent['Entity'][i]+'\n'
  
  if 'APPEAL_NO' in temp:
      if(CASENAME):
          re+=CASENAME+' hasAppealNo '+ tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasAppealNo '+ tok_ent['Entity'][i]+'\n'

  
  if 'AUTHOR' in temp:
      re+=CASENAME+' hasAuthor '+tok_ent['Entity'][i]+'\n'

  if 'BENCH' in temp:
      re+=CASENAME+' hasCourtOfficial Judge '+'\n'
      str=tok_ent['Entity'][i]
      my_list =str.split(",")
      for x in range(len(my_list)):
          re+='Judge hasName '+my_list[x]+'\n'

  if 'JURISDICTION' in temp:
      if(CASENAME):
          re+=CASENAME+' hasJurisdiction '+tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasJurisdiction '+tok_ent['Entity'][i]+'\n'
  
  if 'COURT_TYPE' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCourt '+tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCourt '+tok_ent['Entity'][i]+'\n'
      
      if(tok_ent['Token'][i+1]=='LOCATION'):
          re+=tok_ent['Entity'][i]+' hasLocation '+tok_ent['Entity'][i+1]+'\n'

  if 'COURT_DECISION' in temp:
      if(CASENAME):
          re+=CASENAME+' hasCourtDecision '+tok_ent['Entity'][i]+'\n'
      else:
          re+='Case hasCourtDecision '+tok_ent['Entity'][i]+'\n'
    

In [ ]:
print(re)


 1314.txt
CASE hasCaseName   SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996
  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996 hasParty PETITIONER 
PETITIONER hasName   SHIV SAGAR TIWARI 
  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996 hasParty RESPONDENT 
RESPONDENT hasName   UNION OF INDIA & ORS. 
  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996 hasDate  08/11/1996
  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996 hasDate 1996
  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996 hasDate 11
  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996 hasCaseType  criminal
  SHIV SAGAR TIWARI VS. UNION OF INDIA & ORS.  on 08/11/1996 hasCourtOfficial Judge 
Judge hasName   KULDIP SINGH
Judge hasName  B.L. HANSARIA 

